In [10]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time

In [11]:
raw = pd.read_csv('../../data/original/working_federal_reporter_2020.csv',engine='python')

In [12]:
raw.reset_index(inplace = True)
raw.rename(columns={'index':'original index'}, inplace=True)

In [13]:
# copy raw into "df" object to preserve "raw" for comparison later, etc
df = raw

In [14]:
# Fill NA values in FY.x with FY.y values
df['FY.x'] = df['FY.x'].fillna(df['FY.y'])

# Rename FY.x to just be "FY"
df = df.rename(columns={'FY.x': 'FY'})

In [15]:
# Fill missing values in Project Start/End Date with Budget Start/End Date
df['PROJECT_START_DATE'] = df['PROJECT_START_DATE'].fillna(df['BUDGET_START_DATE'])
df['PROJECT_END_DATE'] = df['PROJECT_END_DATE'].fillna(df['BUDGET_END_DATE'])

#If START date is still missing, fill with FY
df['PROJECT_START_DATE'] = df['PROJECT_START_DATE'].fillna(df['FY'])

In [16]:
#remove rows with NULL abstracts
df = df[~raw.ABSTRACT.isnull()]

In [17]:
# drop abstracts with values of "ABSTRACT NOT PROVIDED" and "No abstract provided "
df = df[df.ABSTRACT != 'ABSTRACT NOT PROVIDED']
df = df[df.ABSTRACT != 'No abstract provided']

#### Aggregate counts for unique ORGANIZATION_NAMEs in rows with duplicated Abstract/Title/Project_Start_Date

In [18]:
# group df by abstract/title/start_date (exact matches)
all_grp = df.groupby(['ABSTRACT', 'PROJECT_TITLE', 'PROJECT_START_DATE'])

# for each unique ABSTRACT/TITLE/START_DATE in df, count unique Organizations
unique_all = all_grp.agg({'ORGANIZATION_NAME' : 'nunique'}) 

#rename column as "count" to be different than original column
unique_all = unique_all.rename(columns={'ORGANIZATION_NAME': 'ORG_COUNT'})

# merge df with "unique_all" to bring in the unique Organization counts for each "duplicate" group
merged1 = df.merge(unique_all, left_on=['ABSTRACT', 'PROJECT_TITLE', 'PROJECT_START_DATE'], right_on=['ABSTRACT', 'PROJECT_TITLE', 'PROJECT_START_DATE'])

#### Aggregate counts for unique PIs in rows with duplicated Abstract/Title/Project_Start_Date

In [19]:
# for each unique ABSTRACT/TITLE/START_DATE in whole df, count unique PIs
unique_pi = all_grp.agg({'CONTACT_PI_PROJECT_LEADER' : 'nunique'}) 

#rename column as "count" to be different than original column
unique_pi = unique_pi.rename(columns={'CONTACT_PI_PROJECT_LEADER': 'PI_COUNT'})

# merge data frame with "unique_all" to bring in the unique PI counts for each "duplicate" group
merged2 = merged1.merge(unique_pi, left_on=['ABSTRACT', 'PROJECT_TITLE', 'PROJECT_START_DATE'], right_on=['ABSTRACT', 'PROJECT_TITLE', 'PROJECT_START_DATE'])

In [20]:
# Sort merged data so that duplicated rows occur in order of earliest to latest END date
merged = merged2.sort_values(['PROJECT_END_DATE'])

In [21]:
# save NON-duplicated rows and the LAST occurrance of duplicated rows
dedup = merged[~merged.duplicated(subset=['ABSTRACT',  'PROJECT_TITLE', 'PROJECT_START_DATE'], keep='last')]

In [22]:
#save final (deduplicated) dataframe as "df" to fit downstream code
df = dedup